# This notebook contains the code required to calibrate pipette tips

## Code required to make Fig. 3c is available in this notebook

### Import necessary modules

In [4]:
import pandas as pd
from openpyxl import load_workbook
from scipy.stats import linregress, ttest_ind
import numpy as np
import sys
from plotly import tools, subplots
import plotly.graph_objs as go
import pickle
import plotly.io as pio
pio.templates.default = "none"

if 'ipykernel' in sys.modules:
    from plotly.offline import init_notebook_mode
    from plotly.offline import iplot as plot
    from IPython.display import HTML
    HTML("""
         <script>
          var waitForPlotly = setInterval( function() {
          if( typeof(window.Plotly) !== "undefined" ){
          MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });
          MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);
          clearInterval(waitForPlotly);}}, 250 );
        </script>
        """
    )
    init_notebook_mode(connected=True)

### Make classes/attributes necessary for calibration

In [5]:
class Calibration(object):
    
    #Calibration class will contain all info about one calibration run for one class of volumes
    
    def __init__(self,**kwargs):
        self.max_systematic_error=None
        self.manual_slope=None
        self.manual_intercept=None
        self.number_tips=8
        self.number_replicates=None
        for key in kwargs:
            setattr(self, key, kwargs[key])
        self.tip_objects={"tip"+str(i+1): None for i in range(self.number_tips)}
        self.tip_objects["manual"]=None
        
        
    def scale_od(self):
        
        #Figures out max and min OD of manually pipetted samples for each replicate separately
        minmax_params={replicate.replicate_id: {'max':max([point.raw_od for point in replicate.points]),
                                                'min':min([point.raw_od for point in replicate.points])}
                           for replicate in self.tip_objects["manual"].replicates}
        
        #Scale the raw_OD of every point between the min and max values for that replicate
        for tip_id, tip in self.tip_objects.items():
            for replicate in tip.replicates:
                if replicate.replicate_id!='solution':
                    for point in replicate.points:
                        point.scaled_od = (point.raw_od - minmax_params[replicate.replicate_id]['min'])/(minmax_params[replicate.replicate_id]['max'] - minmax_params[replicate.replicate_id]['min'])
                 
                
    def calculate_std_curve(self):
        #Calculate std curve that relates OD to volume (and consequently concentration)
        slopes=[]
        intercepts=[]
        for replicate in self.tip_objects["manual"].replicates:
            replicate.calculate_std_curve()
            slopes.append(replicate.slope)
            intercepts.append(replicate.intercept)
        self.man_slope = np.mean(slopes)
        self.man_slope_std = np.std(slopes)
        self.man_intercept = np.mean(intercepts)
        self.man_intercept_std = np.std(intercepts)
    
    
    def predict_volumes(self):
        
        #Go to each point and predict the volume pipetted as the OD*avg_slope+avg_intercept.
        #Also calculate the pct_deviation from expected values
        for tip_id,tip in self.tip_objects.items():
            if tip_id!='manual':
                for replicate in tip.replicates:
                    if replicate.replicate_id!='solution':
                        for point in replicate.points:
                            point.predicted_value = self.man_slope * point.scaled_od + self.man_intercept
                            point.pct_deviation = abs(point.expected_value - point.predicted_value)/point.expected_value*100


    def calculate_calibration_params(self):
        
        #Create a calibration point for each volume with the average predicted value and expected value
        for tip_id, tip in self.tip_objects.items():
            if tip_id!='manual':
                volumes_list = [point.expected_value for point in tip.replicates[0].points]
                tip.replicate_dict['solution'].points.extend([CalibrationPoint(expected_value=volume,
                                                          tip_id=tip_id,
                                                          predicted_value=np.mean([replicate.points_dict[volume].predicted_value 
                                                                                   for replicate in tip.replicates
                                                                                  if replicate.replicate_id!='solution']),
                                                          pct_deviation=np.mean([replicate.points_dict[volume].pct_deviation
                                                                             for replicate in tip.replicates
                                                                             if replicate.replicate_id!='solution']))
                                                         for volume in volumes_list])
                
                #For tip, calculate an offset and a factor that can convert actual pipetted volumes to desired volumes
                #Tecan's EvoWare interface requires tip_factor,tip_offset to be the slope and intercept that transforms
                #  pipetted values(x's) into expected values(y's)
                x_vals=[point.predicted_value for point in tip.replicate_dict['solution'].points]
                y_vals=[point.expected_value for point in tip.replicate_dict['solution'].points]
                sol = linregress(x_vals, y_vals)
                tip.replicate_dict['solution'].tip_factor = sol.slope
                tip.replicate_dict['solution'].tip_offset = sol.intercept
                
                #Calculate rsq to get the feasibility of a linear regression and the average pct deviation for the tip WITHOUT calibration
                tip.replicate_dict['solution'].tip_rsq = sol.rvalue**2
                tip.replicate_dict['solution'].avg_pct_deviation = np.mean([point.pct_deviation for point in tip.replicate_dict['solution'].points])
        
    
    def load_calibration_data(self, filename='None'):
        #Load calibration file
        if filename == 'None':
            print("No filename specified")
        else:
            #Load entire workbook
            wb=load_workbook(filename)
            
            #Find which sheets begin with 'Rep' and calculate number of replicate calibrations done
            self.number_replicates=len([sheetname for sheetname in wb.sheetnames if 'Rep' in sheetname])
            if self.number_replicates==0 or self.number_replicates is None:
                print("Please check sheet names. No sheets with the name Rep to indicate replicates.")
            else:
                
                # Initiate empty Tip object for each tip in the dictionary 'tip_objects'
                for tip_id in self.tip_objects:
                    self.tip_objects[tip_id] = Tip(tip_id=tip_id,
                                                  number_replicates=self.number_replicates)
                    
                #For each replicate, find the number of OD reads (tech replicates to identify random errors)
                #Also, store the index of the row where OD data begins
                for rep in range(self.number_replicates):
                    datasheet = 'Rep'+str(rep+1)
                    data_start_index = [index+5 for index, row in enumerate(wb[datasheet].rows) 
                                        if row[0].value if "Start Time" in str(row[0].value)]
                    number_reads = len(data_start_index)
                    
                    #For each calibration point - i.e. pipetted volume, get expected value from the setup sheet and 
                    # the raw OD from the appropriate replicate sheet
                    for j,column in enumerate(wb['Setup'].iter_cols()):
                        if str(column[0].value).title() != 'Manual':
                            for i in range(8):
                                if column[i+1].value is not None:
                                    #Read error stores the random error/systematic error from the platereader
                                    point = CalibrationPoint(expected_value=column[i+1].value, tip_id=i+1)        
                                    point.raw_od = np.mean([wb[datasheet][index+i][j+1].value for index in data_start_index])
                                    point.read_error = np.std([wb[datasheet][index+i][j+1].value for index in data_start_index])
                                    self.tip_objects["tip"+str(i+1)].replicate_dict[rep].points.append(point)


                        else:
                            #Same as above for manually pipetted values
                            for i in range(8):
                                if column[i+1].value is not None:
                                    point = ManualPoint(expected_value=column[i+1].value, tip_id='manual')
                                    point.raw_od = np.mean([wb[datasheet][index+i][j+1].value for index in data_start_index])
                                    point.read_error = np.std([wb[datasheet][index+i][j+1].value for index in data_start_index])
                                    self.tip_objects["manual"].replicates[rep].points.append(point)
                                    
                #Identify max systematic error from platereader for an estimation of the number of significant figures to display
                self.max_systematic_error = np.amax([max([max([point.read_error for point in replicate.points])
                                                     for replicate in tip_object.replicates
                                                     if replicate.replicate_id!='solution']) 
                                                    for tip_id,tip_object in self.tip_objects.items()])/np.sqrt(number_reads)
                
                #Call functions to scale OD data, calculate std curve for manually pipetted samples, predict volumes, and calculate calibration params
                self.scale_od()
                self.calculate_std_curve()
                self.predict_volumes()
                self.calculate_calibration_params()

    
class Tip(object):
    
    #Each tip object has information about the calibration replicates available for that tip.
    #After all calculations are done, it will also contain the 'solution' replicate which has the offset and factor values
    def __init__(self,**kwargs):
        self.number_replicates=None
        self.tip_id=None
        for key in kwargs:
            setattr(self, key, kwargs[key])
        self.replicates=[Replicate(tip_id=self.tip_id, replicate_id=i)
                         if self.tip_id!="manual"
                         else ManualReplicate(tip_id=self.tip_id, replicate_id=i)
                         for i in range(self.number_replicates)]
        
        if self.tip_id!="manual":
            self.replicates.append(Replicate(tip_id=self.tip_id, replicate_id='solution')) 
        
        
    @property 
    def replicate_dict(self):
        return{replicate.replicate_id:replicate for replicate in self.replicates}

    
class Replicate(object):
    
    #Each Replicate object contains calibration points
    def __init__(self, **kwargs):
        self.points=[]
        self.tip_id=None
        self.replicate_id=None
        for key in kwargs:
            setattr(self, key, kwargs[key])
            
    @property
    def points_dict(self):
        return {point.expected_value:point for point in self.points}
        
    

class ManualReplicate(Replicate):
    
    #Child class of Replicate object. Has the slope, intercept and rsq values for the manually pipetted replicate
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.slope=None
        self.intercept=None
        self.rsq=None
        
        
    def calculate_std_curve(self):
        #Calculate linear regression params that transform OD values to Volume (as a proxy for concentration)
        if self.points:
            x_vals=[point.scaled_od for point in self.points]
            y_vals=[point.expected_value for point in self.points]
            lin_fit = linregress(x_vals, y_vals)
            self.slope=lin_fit.slope
            self.intercept=lin_fit.intercept
            self.rsq=lin_fit.rvalue**2
              
            
class ManualPoint(object):
    
    #Class that contains a calibration point for manually pipetted sample
    def __init__(self,**kwargs):
        self.number_reads=None
        self.expected_value=None
        self.raw_od=None
        self.scaled_od=None
        self.read_error=None
        self.tip_id=None
        for key in kwargs:
            setattr(self, key, kwargs[key])

        
class CalibrationPoint(ManualPoint):
        
    #Child class of Manual Point that contains a calibration point for liha pipetted sample

    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        self.predicted_value=None
        self.pct_deviation=None
        for key in kwargs:
            setattr(self, key, kwargs[key])

### Calibration Performed for 3 Levels:


- Low Volume: 3$\mu$L - 10$\mu$L
- Mid Volume: 10$\mu$L - 50$\mu$L
- High Volume: 50$\mu$L - 200$\mu$L

In [6]:
##Importing files with absorbance data

high_volumes_calib = Calibration()
high_volumes_calib.load_calibration_data('20201002_HighVolume.xlsx')

mid_volumes_calib = Calibration()
mid_volumes_calib.load_calibration_data('20201002_MidVolume.xlsx')

low_volumes_calib = Calibration()
low_volumes_calib.load_calibration_data('20201002_LowVolume.xlsx')


#Due to calibration loss issues with some tips, calibration for the bottom two levels was repeated at a later date.
mid_volumes_calib_new = Calibration()
mid_volumes_calib_new.load_calibration_data('20210215_MidVolume.xlsx')

low_volumes_calib_new = Calibration()
low_volumes_calib_new.load_calibration_data('20210215_LowVolume.xlsx')

### Calculate Calibration Parameters (Slope/Intercept)

In [7]:
#Loading 2 lists for calibration objects.
#Old (2-Oct-2020) Calibration used for High Volume, Tip3 and Tip7 for Mid Volume, and Tip8 for Low Volume
#Others use New (15-Feb-2021) Calibration Values

calib_list = [low_volumes_calib, mid_volumes_calib, high_volumes_calib]
calib_list_new = [low_volumes_calib_new, mid_volumes_calib_new, high_volumes_calib]

titles = ("3\u03BCL - 10\u03BCL", "10\u03BCL - 50\u03BCL", "50\u03BCL - 200\u03BCL")
col_titles = ("low","mid","high")

df_list = []

#Load offsets and factors from calib objects and put them in a pandas dataframe
for i,(calib,calib_new) in enumerate(zip(calib_list, calib_list_new)):
    df_list.append(pd.DataFrame(data={'factor':
                                      np.around([calib.tip_objects[tip_id].replicate_dict['solution'].tip_factor 
                                                 if (tip_id in ['tip3','tip7'] and i==1) or (tip_id=='tip8' and i==0) 
                                                 else
                                                 calib_new.tip_objects[tip_id].replicate_dict['solution'].tip_factor
                                                 for tip_id, tip in calib.tip_objects.items()
                                                 if tip_id!='manual'], decimals=3),
                                      
                                      'offsets':
                                      np.around([calib.tip_objects[tip_id].replicate_dict['solution'].tip_offset 
                                                 if (tip_id in ['tip3','tip7'] and i==1) or (tip_id=='tip8' and i==0) 
                                                 else
                                                 calib_new.tip_objects[tip_id].replicate_dict['solution'].tip_offset
                                                 for tip_id, tip in calib.tip_objects.items()
                                                 if tip_id!='manual'], decimals=3)},
                                
                                index=[tip_id.title() 
                                       for tip_id in low_volumes_calib.tip_objects 
                                       if tip_id!='manual']))
    
    
calib_df = pd.concat(df_list,axis=1,keys=(['low','mid','high']))
display(calib_df)

low            mid           high        
     factor offsets factor offsets factor offsets
Tip1  1.036   0.439  1.060  -0.631  1.032   0.442
Tip2  1.081   0.016  1.071  -0.825  1.041   0.007
Tip3  1.057   0.156  1.010   0.116  1.031  -0.129
Tip4  1.128   0.085  1.085  -0.871  1.037   0.413
Tip5  1.092   0.218  1.081  -0.654  1.038   0.524
Tip6  1.074   0.066  1.078  -0.869  1.028   0.901
Tip7  1.071   0.670  1.013   0.825  1.039   0.312
Tip8  1.014  -0.122  1.090  -1.494  1.023   1.270

### Obtain plots showing pre-calibration pipetted volumes and calibration line

In [12]:
colors = ['rgb(220,50,32)',
          'rgb(236,170,22)',
          'rgb(0,108,209)']

widths = [0.35, 2, 7.5]
ranges = [(2.5,10.5), (7.5, 52.5), (45, 205)]
tickvals = [[3,4,5,6,7,8,9,10],
            [10, 12.5, 15, 20, 25, 30, 40, 50],
            [50, 60, 75, 100, 125, 150, 175, 200]]

xaxis2_ranges = [(0,15),(0,7.5),(0,5)]
xaxis2_tickvals = [[0,3,6,9,12,15],
                   [0,2.5,5,7.5],
                   [0,1,2,3,4,5]]

titles = ("3\u03BCL - 10\u03BCL", "10\u03BCL - 50\u03BCL", "50\u03BCL - 200\u03BCL")

for tip_id in low_volumes_calib.tip_objects:
    
    if tip_id == 'manual':
        continue
    
    for i,(calib,calib_new) in enumerate(zip(calib_list, calib_list_new)):
        
        if (i==0 and tip_id=='tip8') or (i==1 and tip_id in ['tip3','tip7']):
            
            expected_volumes = [point.expected_value 
                                for point in calib.tip_objects[tip_id].replicate_dict['solution'].points]
            actual_volumes = [point.predicted_value 
                              for point in calib.tip_objects[tip_id].replicate_dict['solution'].points]
            deviations = [np.mean([replicate.points_dict[expected_value].pct_deviation 
                                   for replicate in calib.tip_objects[tip_id].replicates 
                                   if replicate.replicate_id!='solution'])
                          for expected_value in expected_volumes]
            err_deviations = [np.std([replicate.points_dict[expected_value].pct_deviation 
                                      for replicate in calib.tip_objects[tip_id].replicates 
                                      if replicate.replicate_id!='solution'])/np.sqrt(calib.number_replicates)
                              for expected_value in expected_volumes]
            
            calib_trace = go.Scatter(x=np.arange(0,250), 
                                     y=calib.tip_objects[tip_id].replicate_dict['solution'].tip_factor *
                                     np.arange(0,250) + calib.tip_objects[tip_id].replicate_dict['solution'].tip_offset,
                                     name='Calibration Fit', mode='lines', line=dict(width=2, color=colors[i]),
                                     showlegend=False)
        else:

            expected_volumes = [point.expected_value 
                                for point in calib_new.tip_objects[tip_id].replicate_dict['solution'].points]
            actual_volumes = [point.predicted_value 
                              for point in calib_new.tip_objects[tip_id].replicate_dict['solution'].points]
            deviations = [np.mean([replicate.points_dict[expected_value].pct_deviation 
                                   for replicate in calib_new.tip_objects[tip_id].replicates 
                                   if replicate.replicate_id!='solution'])
                          for expected_value in expected_volumes]
            err_deviations = [np.std([replicate.points_dict[expected_value].pct_deviation 
                                      for replicate in calib_new.tip_objects[tip_id].replicates 
                                      if replicate.replicate_id!='solution'])/np.sqrt(calib_new.number_replicates)
                              for expected_value in expected_volumes]
            
            calib_trace = go.Scatter(x=np.arange(0,250), 
                                     y=calib_new.tip_objects[tip_id].replicate_dict['solution'].tip_factor * 
                                     np.arange(0,250) + 
                                     calib_new.tip_objects[tip_id].replicate_dict['solution'].tip_offset,
                                     name='Calibration Line', mode='lines', line=dict(width=2, color=colors[i]),
                                     showlegend=True)
            
        experimental_trace =go.Scatter(x=actual_volumes,y=expected_volumes,
                                       name='Calibration Points<br>(Primary X)', mode = 'markers',
                                       marker_color=colors[i])
        expected_trace = go.Scatter(x=np.arange(0,250),y=np.arange(0,250),
                                    name='Expected Volume', mode = 'lines',
                                    line=dict(dash='dot', color='black', width=1), showlegend=False)
        dev_trace = go.Bar(y=expected_volumes, x=deviations,
                           error_x=dict(type='data', array=err_deviations, width=2, thickness=1, color='black'),
                           name='% Error<br>(Secondary X)',
                           marker=dict(color=colors[i], line=dict(width=0)),
                           orientation='h',  xaxis='x2', opacity=0.35, width=widths[i])
        
        layout = go.Layout(height=350, width=410,
                           titlefont=dict(family='Arial', size=14, color='black'),
                           title_yanchor='bottom', title_xanchor='center', title_pad={'t':50,'b':150},
                           title_x=0.15, title_y=0.5,
                           
                           xaxis=dict(title='Pipetted Volume (\u03BCL)', title_standoff=0,
                                      titlefont=dict(family='Arial', size=14, color='black'),
                                      showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                                      ticks='outside', ticklen=4, tickangle=0, nticks=5,
                                      tickfont=dict(size=13, family='Arial', color='black'), tickcolor='black',
                                      showgrid=False, range=ranges[i]),
                           
                           xaxis2=dict(title='Avg % Error',  title_standoff=0,
                                       titlefont=dict(family='Arial', size=14, color='black'),
                                       showline=True, linewidth=1, linecolor='black', mirror=False, side='top',
                                       anchor='y', overlaying='x',
                                       ticks='outside', ticklen=4, tickangle=0, tickvals=xaxis2_tickvals[i],
                                       tickfont=dict(family='Arial',size=13, color='black'), tickcolor='black',
                                       showgrid=False, range=xaxis2_ranges[i]),
                           
                           yaxis=dict(title='Programmed Volume (\u03BCL)', title_standoff=0,
                                      titlefont=dict(family='Arial', size=14, color='black'),
                                      showline=True, linewidth=1, linecolor='black', mirror=True,
                                      ticks='outside', ticklen=4, tickangle=0, tickvals=tickvals[i],
                                      tickfont=dict(family='Arial',size=13, color='black'), tickcolor='black', 
                                      showgrid=False, range=ranges[i]))
        
        fig = go.Figure(data=[experimental_trace,  dev_trace, calib_trace, expected_trace,], layout=layout)
        
        if tip_id=='tip4':
            plot(fig)


            #pio.write_image(fig,Figures/fig_2_airgapeffect"+col_titles[i]+".svg",format='svg')


### Load pipette test data

In [13]:
#These files contain absorbance data from tests run after calibrating the pipettes

post_calib_high_volumes=Calibration()
post_calib_high_volumes.load_calibration_data('20201004_Test_HighVolume.xlsx')

post_calib_mid_volumes=Calibration()
post_calib_mid_volumes.load_calibration_data('20201004_Test_MidVolume.xlsx')

post_calib_low_volumes=Calibration()
post_calib_low_volumes.load_calibration_data('20201004_Test_LowVolume.xlsx')

post_calib_mid_volumes_new=Calibration()
post_calib_mid_volumes_new.load_calibration_data('20210215_Test_MidVolume.xlsx')

post_calib_low_volumes_new=Calibration()
post_calib_low_volumes_new.load_calibration_data('20210215_Test_LowVolume.xlsx')

post_calib_list=[post_calib_low_volumes, post_calib_mid_volumes, post_calib_high_volumes]
post_calib_list_new=[post_calib_low_volumes_new, post_calib_mid_volumes_new, post_calib_high_volumes]


### Plot post-calibration pipette efficacy

In [14]:
colors = ['rgb(220,50,32)',
          'rgb(236,170,22)',
          'rgb(0,108,209)']

widths = [0.35, 2, 7.5]
ranges = [(2.5,10.5), (7.5, 52.5), (45, 205)]
tickvals = [[3,4,5,6,7,8,9,10],
            [10, 12.5, 15, 20, 25, 30, 40, 50],
            [50, 60, 75, 100, 125, 150, 175, 200]]

xaxis2_ranges = [(0,15),(0,7.5),(0,5)]
xaxis2_tickvals = [[0,3,6,9,12,15],
                   [0,2.5,5,7.5],
                   [0,1,2,3,4,5]]

titles = ("3\u03BCL - 10\u03BCL", "10\u03BCL - 50\u03BCL", "50\u03BCL - 200\u03BCL")

for tip_id in low_volumes_calib.tip_objects:
    
    if tip_id == 'manual':
        continue
    
    for i,(calib,calib_new) in enumerate(zip(post_calib_list, post_calib_list_new)):
        
        if (i==0 and tip_id=='tip8') or (i==1 and tip_id in ['tip3','tip7']):
            
            expected_volumes = [point.expected_value 
                                for point in calib.tip_objects[tip_id].replicate_dict['solution'].points]
            actual_volumes = [point.predicted_value 
                              for point in calib.tip_objects[tip_id].replicate_dict['solution'].points]
            deviations = [np.mean([replicate.points_dict[expected_value].pct_deviation 
                                   for replicate in calib.tip_objects[tip_id].replicates 
                                   if replicate.replicate_id!='solution'])
                          for expected_value in expected_volumes]
            
        else:

            expected_volumes = [point.expected_value 
                                for point in calib_new.tip_objects[tip_id].replicate_dict['solution'].points]
            actual_volumes = [point.predicted_value 
                              for point in calib_new.tip_objects[tip_id].replicate_dict['solution'].points]
            deviations = [np.mean([replicate.points_dict[expected_value].pct_deviation 
                                   for replicate in calib_new.tip_objects[tip_id].replicates 
                                   if replicate.replicate_id!='solution'])
                          for expected_value in expected_volumes]
            
        experimental_trace =go.Scatter(x=actual_volumes,y=expected_volumes,
                                       name='Calibration Points<br>(Primary X)', mode = 'markers',
                                       marker_color=colors[i])
        expected_trace = go.Scatter(x=np.arange(0,250),y=np.arange(0,250),
                                    name='Expected Volume', mode = 'lines',
                                    line=dict(dash='dot', color='black', width=1), showlegend=False)
        dev_trace = go.Bar(y=expected_volumes, x=deviations,
                           name='% Error<br>(Secondary X)',
                           marker=dict(color=colors[i], line=dict(width=0)),
                           orientation='h',  xaxis='x2', opacity=1, width=widths[i])
        
        layout = go.Layout(height=350, width=425,
                           title=tip_id.title()+"_"+str(col_titles[i])+"<br>"+titles[i],
                           titlefont=dict(family='Arial', size=14, color='black'),
                           title_yanchor='bottom', title_xanchor='center', title_pad={'t':50,'b':150},
                           title_x=0.15, title_y=0.5,
                           
                           xaxis=dict(title='Pipetted Volume (\u03BCL)', title_standoff=0,
                                      titlefont=dict(family='Arial', size=14, color='black'),
                                      showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                                      ticks='outside', ticklen=4, tickangle=0, nticks=5,
                                      tickfont=dict(size=13, family='Arial', color='black'), tickcolor='black',
                                      showgrid=False, range=ranges[i]),
                           
                           xaxis2=dict(title='Avg % Error',  title_standoff=0,
                                       titlefont=dict(family='Arial', size=14, color='black'),
                                       showline=True, linewidth=1, linecolor='black', mirror=False, side='top',
                                       anchor='y', overlaying='x',
                                       ticks='outside', ticklen=4, tickangle=0, tickvals=xaxis2_tickvals[i],
                                       tickfont=dict(family='Arial',size=13, color='black'), tickcolor='black',
                                       showgrid=False, range=xaxis2_ranges[i]),
                           
                           yaxis=dict(title='Programmed Volume (\u03BCL)', title_standoff=0,
                                      titlefont=dict(family='Arial', size=14, color='black'),
                                      showline=True, linewidth=1, linecolor='black', mirror=True,
                                      ticks='outside', ticklen=4, tickangle=0, tickvals=tickvals[i],
                                      tickfont=dict(family='Arial',size=13, color='black'), tickcolor='black', 
                                      showgrid=False, range=ranges[i]))
        
        fig = go.Figure(data=[experimental_trace,  dev_trace, expected_trace,], layout=layout)
        
        if tip_id=='tip4':
            plot(fig)

#if not os.path.exists('images'):
#    os.mkdir('images')
#pio.write_image(fig, 'images/fold_improvement_objB.svg')    

### Determine whether calibration was successful.
#### Obtain average %error for each tip in every volume level.
#### Perform a one-tailed t-test with the alternative hypothesis : Post-calibration error is smaller than Pre-calibration error

In [15]:
df_list = []
for i, ((calib,post_calib), (calib_new,post_calib_new)) in enumerate(zip(zip(calib_list,post_calib_list), zip(calib_list_new, post_calib_list_new))):
        
    prior_error = []
    prior_error_stderr = []
    post_error = []
    post_error_stderr = []
    pvalue = []
    
    for tip_id in calib.tip_objects:
        
        if tip_id!='manual':
            
            if (i==0 and tip_id=='tip8') or (i==1 and tip_id in ['tip3','tip7']):
                
                temp_priors = np.array([[point.pct_deviation for point in replicate.points] 
                                         for replicate in calib.tip_objects[tip_id].replicates
                                         if replicate.replicate_id!='solution']).flatten()
                temp_posts = np.array([[point.pct_deviation for point in replicate.points] 
                                        for replicate in post_calib.tip_objects[tip_id].replicates
                                        if replicate.replicate_id!='solution']).flatten()
            else:
                temp_priors = np.array([[point.pct_deviation for point in replicate.points] 
                                         for replicate in calib_new.tip_objects[tip_id].replicates
                                         if replicate.replicate_id!='solution']).flatten()
                temp_posts = np.array([[point.pct_deviation for point in replicate.points] 
                                        for replicate in post_calib_new.tip_objects[tip_id].replicates
                                        if replicate.replicate_id!='solution']).flatten()
                                                                    
            prior_error.append(np.mean(temp_priors))
            prior_error_stderr.append(np.std(temp_priors)/np.sqrt(len(temp_priors)))
            post_error.append(np.mean(temp_posts))
            post_error_stderr.append(np.std(temp_posts)/np.sqrt(len(temp_posts)))
            pvalue.append(ttest_ind(temp_posts,temp_priors,equal_var=False,alternative='less').pvalue)

    
    df_list.append(pd.DataFrame(data={'pre_error':prior_error,
                                      'pre_error_stderr':prior_error_stderr,
                                      'post_error': post_error,
                                      'post_error_stderr':post_error_stderr,
                                      'pvalue':pvalue},
                                index=[tip_id.title() for tip_id in calib.tip_objects if tip_id!='manual']))
    
calib_result_df = pd.concat(df_list, axis=1, keys=(['low','mid','high']))
display(calib_result_df)

low                                                              \
      pre_error pre_error_stderr post_error post_error_stderr        pvalue   
Tip1  10.691489         0.826060   2.448278          1.542980  4.994176e-04   
Tip2   7.722152         0.497169   4.255084          1.193549  1.584805e-02   
Tip3   7.987718         0.545860   5.154464          1.464145  6.147275e-02   
Tip4  12.659403         0.468757   3.748540          0.889757  2.419400e-06   
Tip5  11.681890         0.653823   2.723800          0.813242  2.101258e-07   
Tip6   7.868794         0.471157   2.959927          1.322828  4.955371e-03   
Tip7  17.547179         0.930633   4.430812          2.701350  1.093077e-03   
Tip8   5.019292         0.963276   4.204163          1.430452  3.305915e-01   

           mid                                                              \
     pre_error pre_error_stderr post_error post_error_stderr        pvalue   
Tip1  2.773798         0.292254   1.028369          0.145962  7.014732e-06   
Tip2  3.336773         0.278230   1.177206          0.163008  1.927289e-07   
Tip3  1.918661         0.303597   1.367043          0.370150  1.442217e-01   
Tip4  3.776630         0.459559   1.378770          0.290132  9.684151e-05   
Tip5  4.382359         0.383428   1.783168          0.259298  4.251480e-06   
Tip6  3.640263         0.383495   1.439206          0.374395  3.845441e-04   
Tip7  5.367916         0.485163   1.089328          0.440428  1.174245e-06   
Tip8  4.293257         0.473823   1.393048          0.407348  8.396812e-05   

          high                                                              
     pre_error pre_error_stderr post_error post_error_stderr        pvalue  
Tip1  3.534503         0.092623   0.603856          0.180391  2.342212e-08  
Tip2  3.946214         0.097624   0.547632          0.152522  1.218752e-10  
Tip3  2.868561         0.115967   0.418166          0.126529  2.092210e-11  
Tip4  4.020667         0.129134   0.914252          0.537572  4.192904e-04  
Tip5  4.182377         0.104530   0.415334          0.099864  1.204527e-17  
Tip6  3.669174         0.121306   0.502977          0.123842  7.284577e-14  
Tip7  4.006172         0.139171   0.616360          0.145403  5.248857e-13  
Tip8  3.510751         0.174482   0.682338          0.161568  4.796618e-11

### Plot average errors pre- and post-calibration

In [16]:
colors = [['#379244','#B5DCB5'],
          ['#379244','#B5DCB5'],
          ['#379244','#B5DCB5']]

yaxis_ranges = [(0, 20),
                (0, 6),
                (0, 5)]

#These are max acceptable error levels laid out by ISO for each volume level
iso_ranges = [(2.4, 8),
              (2.2, 4),
              (1.6, 2)]

#Typically found max errors for manual multichannel pipettes
manual_ranges = [(1, 5.5),
                 (0.8, 2.5),
                 (0.8, 1.2)]
levels = ['low','mid','high']
xaxis_titles = [levels[0].title()+"<br>(3\u03BCL - 10\u03BCL)"]+[levels[1].title()+"<br>(10\u03BCL - 50\u03BCL)"]+[levels[2].title()+"<br>(50\u03BCL - 200\u03BCL)"]

for i, level in enumerate(['low','mid','high']):
    trace_list = []
    trace_list.append(go.Scatter(x=np.linspace(0, 10, 8), y=[manual_ranges[i][0]]*8,
                                  mode='lines', line=dict(color='grey', width=0),
                                 xaxis='x2', showlegend=False))
    trace_list.append(go.Scatter(x=np.linspace(0, 10, 8), y=[manual_ranges[i][1]]*8,
                                 mode='lines', line=dict(color='grey',width=0),
                                 fill='tonexty', fillcolor='rgba(0,0,0,0.2)',
                                 xaxis='x2', showlegend=False))
    trace_list.append(go.Scatter(x=np.linspace(0, 10, 8), y=[iso_ranges[i][0]]*8,
                                 xaxis='x2', line=dict(dash='dot',color='black',width=0.5),
                                 mode='lines', showlegend=False, visible=False))
    trace_list.append(go.Scatter(x=np.linspace(0, 10, 8), y=[iso_ranges[i][1]]*8,
                                 mode='lines', line=dict(dash='dot',color='#9b5151',width=0.5),
                                 xaxis='x2', showlegend=False))
    trace_list.append(go.Bar(x=list(calib_result_df.index), y=calib_result_df[level]['pre_error'].to_list(),
                             error_y=dict(type='data', array=calib_result_df[level]['pre_error_stderr'].to_list(),
                                          width=2, thickness=1, color='black'),
                             marker=dict(color=colors[i][0], line=dict(width=0, color='black')),
                             textfont=dict(family='Arial', size=20, color='black'),
                             name='Average Pre-Calibration Error', showlegend=False))
    trace_list.append(go.Bar(x=list(calib_result_df.index), y=calib_result_df[level]['post_error'].to_list(),
                             error_y=dict(type='data', array=calib_result_df[level]['post_error_stderr'].to_list(),
                                          width=2, thickness=1, color='black'),
                             marker=dict(color=colors[i][1], line=dict(width=0, color='black')),
                             textfont=dict(family='Arial', size=40, color='black'),
                             name='Average Post-Calibration Error', showlegend=False))
    
    layout = go.Layout(height=450, width=425, showlegend=True, barmode='group', bargap=0.2, bargroupgap=0.05,
                       
                       xaxis=dict(
                                  title_font=dict(family='Arial',size=13,color='black'), title_standoff=0,
                                  showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False,
                                  ticks='outside', ticklen=4, tickfont=dict(size=13, family='Arial', color='black'),
                                  tickangle=0, tickcolor='black', side='bottom', 
                                  type='category', anchor='y', overlaying='x2'),

                       xaxis2=dict(showline=False,linewidth=0,ticklen=0,side='top',type='linear',range=(0,8),
                                   anchor='y',visible=False),
                           
                       yaxis=dict(title='Pipetting Accuracy<br>(% deviation)',
                                  titlefont=dict(family='Arial', size=16, color='black'), title_standoff=0,
                                  showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False,
                                  ticks='outside', ticklen=4, tickfont=dict(family='Arial',size=13, color='black'),
                                  tickangle=0, tickcolor='black', range=yaxis_ranges[i]))

    fig = go.Figure(data=trace_list,layout=layout)
    #pio.write_image(fig,"Figures/fig_2_calibration"+level+".svg",format='svg')
    plot(fig)

In [17]:
bar_colors = ['#379244','#B5DCB5']

levels = ['low','mid','high']

x = [[levels[0].title()+"<br>(3\u03BCL - 10\u03BCL)"]*8+
     [levels[1].title()+"<br>(10\u03BCL - 50\u03BCL)"]*8+
     [levels[2].title()+"<br>(50\u03BCL - 200\u03BCL)"]*8,
     [text.replace('Tip','') for text in list(calib_result_df.index)]*3]

trace_list = []


trace_list.append(go.Scatter(x=np.linspace(0, 3.33, 8), y=[manual_ranges[0][0]]*8, 
                             mode='lines', line=dict(color='grey',width=0),
                             xaxis='x2', showlegend=False))
trace_list.append(go.Scatter(x=np.linspace(0, 3.33, 8), y=[manual_ranges[0][1]]*8,
                             mode='lines', line=dict(color='grey',width=0),
                             fill='tonexty',fillcolor='rgba(0,0,0,0.2)',
                             xaxis='x2',showlegend=False))
trace_list.append(go.Scatter(x=np.linspace(3.33, 6.67, 8), y=[manual_ranges[1][0]]*8, 
                             mode='lines', line=dict(color='grey',width=0),
                             xaxis='x2', showlegend=False))
trace_list.append(go.Scatter(x=np.linspace(3.33, 6.67, 8), y=[manual_ranges[1][1]]*8,
                             mode='lines', line=dict(color='grey',width=0),
                             fill='tonexty',fillcolor='rgba(0,0,0,0.2)',
                             xaxis='x2',showlegend=False))
trace_list.append(go.Scatter(x=np.linspace(6.67, 10, 8), y=[manual_ranges[2][0]]*8,
                             mode='lines', line=dict(color='grey',width=0),
                             xaxis='x2', showlegend=False))
trace_list.append(go.Scatter(x=np.linspace(6.67, 10, 8), y=[manual_ranges[2][1]]*8, 
                             name='Reported Accuracy<br>(manual multi-channel pipettes)',
                             mode='lines', line=dict(color='grey',width=0),
                             fill='tonexty', fillcolor='rgba(0,0,0,0.2)',
                             xaxis='x2', showlegend=True))

trace_list.append(go.Scatter(x=np.linspace(0, 3.33, 8), y=[iso_ranges[0][0]]*8,
                             mode='lines', line=dict(dash='dot',color='#9b5151',width=1),
                             xaxis='x2',showlegend=False, visible=False))
trace_list.append(go.Scatter(x=np.linspace(0, 3.33, 8), y=[iso_ranges[0][1]]*8,
                             mode='lines', line=dict(dash='dot',color='#9b5151',width=1),
                             xaxis='x2',showlegend=False))
trace_list.append(go.Scatter(x=np.linspace(3.33, 6.67, 8), y=[iso_ranges[1][0]]*8,
                             mode='lines', line=dict(dash='dot',color='#9b5151',width=1),
                             xaxis='x2',showlegend=False, visible=False))
trace_list.append(go.Scatter(x=np.linspace(3.33, 6.67, 8), y=[iso_ranges[1][1]]*8,
                             mode='lines', line=dict(dash='dot',color='#9b5151',width=1),
                             xaxis='x2',showlegend=False))
trace_list.append(go.Scatter(x=np.linspace(6.67, 10, 8), y=[iso_ranges[2][0]]*8,
                             mode='lines', line=dict(dash='dot',color='#9b5151',width=1),
                             xaxis='x2',showlegend=False, visible=False))
trace_list.append(go.Scatter(x=np.linspace(6.67, 10, 8), y=[iso_ranges[2][1]]*8,
                             name='Max Allowed Error (ISO)',
                             mode='lines', line=dict(dash='dot',color='#9b5151',width=1),
                             xaxis='x2',showlegend=True))

trace_list.append(go.Bar(x=x,
                         y=np.array([calib_result_df[level]['pre_error'].to_list() 
                                     for level in levels]).flatten(),
                         error_y=dict(type='data',
                                      array=np.array([calib_result_df[level]['pre_error_stderr'].to_list() 
                                                      for level in levels]).flatten(),
                                      visible=True, width=2, thickness=1, color='black'),
                         marker=dict(color=bar_colors[0], line=dict(width=0, color='black')),
                         name='Pre-Calibration', 
                         textfont=dict(family='Arial', size=20, color='black'), showlegend=True))

trace_list.append(go.Bar(x=x,
                         y=np.array([calib_result_df[level]['post_error'].to_list() 
                                     for level in levels]).flatten(),
                         error_y=dict(type='data',
                                      array=np.array([calib_result_df[level]['post_error_stderr'].to_list() 
                                                      for level in levels]).flatten(),
                                      visible=True,width=2, thickness=1, color='black'),
                         marker=dict(color=bar_colors[1], line=dict(width=0, color='black')),
                         name='Post-Calibration',
                         textfont=dict(family='Arial', size=40, color='black'),showlegend=True))

layout = go.Layout(height=450, width=950, showlegend=True, legend_orientation='v',
                   barmode='group', bargap=0.4, bargroupgap=0.05, legend_x=0.5, legend_y=0.9,
                   
                   xaxis=dict(showline=True, linewidth=1, linecolor='black', mirror=True, 
                              ticks='outside', ticklen=4, tickangle=0, tickcolor='black',
                              tickfont=dict(size=13, family='Arial', color='black'),
                              side='bottom', showgrid=False, anchor='y', overlaying='x2'),
                   
                   xaxis2=dict(showline=False, linewidth=0, ticklen=0, side='top', type='linear', range=(0,10),
                               anchor='y',visible=False),
                           
                   yaxis=dict(title='Pipetting Accuracy<br>(% deviation)', 
                              title_standoff=0,range=(0,20), titlefont=dict(family='Arial', size=16, color='black'),
                              showline=True, linewidth=1, linecolor='black',mirror=True,
                              ticks='outside', ticklen=4, tickangle=0, tickcolor='black', 
                              tickfont=dict(family='Arial',size=13, color='black'),
                              showgrid=False))

fig = go.Figure(data=trace_list,layout=layout)
fig.update_layout(barmode='group')
#pio.write_image(fig,"Figures/fig_2_final_calib.svg",format='svg')

plot(fig)
display(calib_result_df.loc[:, [('low', 'pvalue'), ('mid', 'pvalue'), ('high', 'pvalue')]])

,low,mid,high
,pvalue,pvalue,pvalue
Tip1,4.994176e-04,7.014732e-06,2.342212e-08
Tip2,1.584805e-02,1.927289e-07,1.218752e-10
Tip3,6.147275e-02,1.442217e-01,2.092210e-11
Tip4,2.419400e-06,9.684151e-05,4.192904e-04
Tip5,2.101258e-07,4.251480e-06,1.204527e-17
Tip6,4.955371e-03,3.845441e-04,7.284577e-14
Tip7,1.093077e-03,1.174245e-06,5.248857e-13
Tip8,3.305915e-01,8.396812e-05,4.796618e-11
